In [8]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score , hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import resample
import numpy as np
import pandas as pd



In [9]:
class MyDecisionTreeClassifier:
    def __init__(self):
        self.model = DecisionTreeClassifier()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)
    
class MyLogisticClassifier:
    def __init__(self, num_classes=9, learning_rate=0.005, num_epochs=5000):
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = None
        self.bias = None

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        m = y_pred.shape[0]
        return -np.sum(np.log(y_pred) * y_true) / m

    def fit(self, X_train, y_train):
        m, n = X_train.shape
        self.weights = np.zeros((n, self.num_classes))
        self.bias = np.zeros((1, self.num_classes))

        for epoch in range(self.num_epochs):
            z = np.dot(X_train, self.weights) + self.bias
            y_pred = self.softmax(z)

            y_train_encoded = np.eye(self.num_classes)[y_train]  # One-hot encoding
            loss = self.cross_entropy_loss(y_pred, y_train_encoded)

            dz = y_pred - y_train_encoded
            dw = np.dot(X_train.T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        y_pred = self.softmax(z)
        return np.argmax(y_pred, axis=1)

class MyMLPClassifier(object):
    def __init__(self, learning_rate=0.001,epochs = 5000, inputLayerSize: int = 12, hiddenLayerSize = [21] , outputLayerSize: int = 9):
        self.inputLayerSize = inputLayerSize
        self.outputLayerSize = outputLayerSize
        self.hiddenLayerSize = hiddenLayerSize
        self.activation = self.sigmoid
        self.activationPrime = self.sigmoidPrime
        self.learning_rate = learning_rate
        self.num_layers = len(hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        self.optimizer = 'SGD'
        self.max_iterations = epochs

    def forward(self, X):
        self.z = [None] * (len(self.weights))
        self.a = [None] * (len(self.weights)-1)
        
        self.z[0] = np.dot(X, self.weights[0])
        for i in range(0,len(self.a)):
            self.a[i] = self.activation(self.z[i])
            self.z[i+1] = np.dot(self.a[i],self.weights[i+1])
        yHat = self.softmax(self.z[-1])
        return yHat

    def relu(self, z):
        return np.maximum(0, z)  # ReLU activation

    def tanh(self, z):
        return np.tanh(z)  # Tanh activation

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoidPrime(self, z):
        return np.exp(-z) / ((1 + np.exp(-z))**2)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z))  # Subtracting max(z) for numerical stability
        return exp_z / exp_z.sum(axis=1, keepdims=True)

    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        J = -np.sum(y * np.log(self.yHat)) / len(X)
        return J

    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)
        deltas = [None] * (len(self.weights))
        deltas[-1] = self.yHat - y
        dJdWs = [None] * (len(self.weights))
        for i in range(len(deltas)-2,-1,-1):
            dJdWs[i+1] = np.dot(self.a[i].T, deltas[i+1])
            deltas[i] = np.dot(deltas[i+1],self.weights[i+1].T)*self.activationPrime(self.z[i])
        dJdWs[0] = np.dot(X.T, deltas[0])
        return dJdWs

    def reluPrime(self, z):
        return np.where(z > 0, 1, 0)  # Derivative of ReLU

    def tanhPrime(self, z):
        return 1 - np.tanh(z)**2  # Derivative of Tanh

    def backward(self, X, y, learning_rate):
        dJdWs = self.costFunctionPrime(X, y)
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * dJdWs[i]
    def fit(self, X, y, batch_size = 32, learning_rate=0.1, max_iterations=10000,wand = 0):
        if(self.optimizer == 'SGD'):
            return self.train_sgd(X=X, y=y,wand=wand)
        elif(self.optimizer == 'Batch'):
            return self.train_batch(X=X, y=y,wand=wand)
        elif(self.optimizer == 'MiniBatch'):
            return self.train_mini_batch(X=X, y=y,wand=wand)
    def train_sgd(self, X, y, learning_rate=0.1, max_iterations=10000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on MLP Classifier for WineQT Dataset")
        for i in range(self.max_iterations):
            idx = np.random.randint(len(X))
            self.backward(X[idx:idx+1], y[idx:idx+1], self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_batch(self, X, y, learning_rate=0.0001, max_iterations=10000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on MLP Classifier for WineQT Dataset")
        for i in range(self.max_iterations):
            self.backward(X, y, self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_mini_batch(self, X, y, batch_size=32, learning_rate=0.01, max_iterations=5000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on MLP Classifier for WineQT Dataset")
        for i in range(self.max_iterations):
            indices = np.arange(len(X))
            np.random.shuffle(indices)
            for j in range(0, len(X), batch_size):
                batch_indices = indices[j:j+batch_size]
                self.backward(X[batch_indices], y[batch_indices], self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")

    def predict(self, X):
        yHat = self.forward(X)
        # return yHat
        binary_predictions = np.zeros_like(yHat)
        binary_predictions[np.arange(len(yHat)), yHat.argmax(axis=1)] = 1
        return binary_predictions
        # return np.argmax(yHat, axis=1)
    def set_learning_rate(self, learning_rate):
        self.learning_rate = learning_rate

    def set_activation_function(self, activation_function):
        if(activation_function == 'sigmoid'):
            self.activation = self.sigmoid
            self.activationPrime = self.sigmoidPrime
        if(activation_function == 'relu'):
            self.activation = self.relu
            self.activationPrime = self.reluPrime
        if(activation_function == 'tanh'):
            self.activation = self.tanh
            self.activationPrime = self.tanhPrime

    def set_optimizer(self, optimizer):
        self.optimizer = optimizer

    def set_hidden_layers(self, hidden_layer_sizes):
        self.hiddenLayerSize = hidden_layer_sizes
        self.num_layers = len(self.hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        # self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
        # self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
        # self.weights, self.biases = self.initialize_weights()
    def set_epochs(self, epcohs):
        self.max_iterations = epochs
        print(self.max_iterations)


In [10]:
class MyDecisionTreeRegressor:
    def __init__(self):
        self.model = DecisionTreeRegressor(random_state=42)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)
    
class MyLinearRegressor:
    def __init__(self):
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0
        # print(X.T)
        # Closed-form solution for linear regression
        X_transpose = X.T
        X = np.array(X)
        y = np.array(y)
        # print(y.shape)
        # print(X.shape)
        # print(X_transpose)
        self.weights = np.dot(np.linalg.inv(np.dot(X_transpose, X)), np.dot(X_transpose, y))
        # print(self.weights)
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return linear_model
    
class MyMLPRegressor(object):
    def __init__(self, learning_rate=0.001,epochs = 5000, inputLayerSize: int = 13, hiddenLayerSize = [13,13,13,13,13,13,13,13,13] , outputLayerSize: int = 1):
        self.inputLayerSize = inputLayerSize
        self.outputLayerSize = outputLayerSize
        self.hiddenLayerSize = hiddenLayerSize
        self.activation = self.sigmoid
        self.activationPrime = self.sigmoidPrime
        self.learning_rate = learning_rate
        # Weights (parameters)
        self.num_layers = len(hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        self.optimizer = 'MiniBatch'
        self.max_iterations = epochs

    def forward(self, X):
        self.z = [None] * (len(self.weights))
        self.a = [None] * (len(self.weights) - 1)

        self.z[0] = np.dot(X, self.weights[0])
        for i in range(0, len(self.a)):
            self.a[i] = self.activation(self.z[i])
            self.z[i + 1] = np.dot(self.a[i], self.weights[i + 1])
        yHat = self.z[-1]  # Linear activation for regression
        return yHat

    def relu(self, z):
        return np.maximum(0, z)  # ReLU activation

    def tanh(self, z):
        return np.tanh(z)  # Tanh activation

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoidPrime(self, z):
        return np.exp(-z) / ((1 + np.exp(-z))**2)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z))  # Subtracting max(z) for numerical stability
        return exp_z / exp_z.sum(axis=1, keepdims=True)

    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        J = np.sum((y - self.yHat) ** 2) / (2 * len(X))  # Mean Squared Error
        return J

    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)
        deltas = [None] * (len(self.weights))
        deltas[-1] = -(y - self.yHat)
        dJdWs = [None] * (len(self.weights))
        for i in range(len(deltas) - 2, -1, -1):
            dJdWs[i + 1] = np.dot(self.a[i].T, deltas[i + 1])
            deltas[i] = np.dot(deltas[i + 1], self.weights[i + 1].T) * self.activationPrime(self.z[i])
        dJdWs[0] = np.dot(X.T, deltas[0])
        return dJdWs

    def reluPrime(self, z):
        return np.where(z > 0, 1, 0)  # Derivative of ReLU

    def tanhPrime(self, z):
        return 1 - np.tanh(z)**2  # Derivative of Tanh

    def backward(self, X, y, learning_rate):
        dJdWs = self.costFunctionPrime(X, y)
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * dJdWs[i]
        # self.W1 -= learning_rate * dJdW1
        # self.W2 -= learning_rate * dJdW2
    def fit(self, X, y, batch_size = 32, learning_rate=0.1, max_iterations=10000,wand = 0):
        if(self.optimizer == 'SGD'):
            return self.train_sgd(X=X, y=y,wand=wand)
        elif(self.optimizer == 'Batch'):
            return self.train_batch(X=X, y=y,wand=wand)
        elif(self.optimizer == 'MiniBatch'):
            return self.train_mini_batch(X=X, y=y,wand=wand)
    def train_sgd(self, X, y, learning_rate=0.1, max_iterations=10000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on Housing Dataset in MLP Regression")
        for i in range(self.max_iterations):
            idx = np.random.randint(len(X))
            self.backward(X[idx:idx+1], y[idx:idx+1], self.learning_rate)
            if i % 100 == 0:
                loss = self.costFunction(X, y)
                print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_batch(self, X, y, learning_rate=0.0001, max_iterations=10000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on Housing Dataset in MLP Regression")
        for i in range(self.max_iterations):
            self.backward(X, y, self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_mini_batch(self, X, y, batch_size=32, learning_rate=0.01, max_iterations=5000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on Housing Dataset in MLP Regression")
        for i in range(self.max_iterations):
            # Randomly shuffle the data and split into mini-batches
            indices = np.arange(len(X))
            np.random.shuffle(indices)
            for j in range(0, len(X), batch_size):
                batch_indices = indices[j:j+batch_size]
                self.backward(X[batch_indices], y[batch_indices], self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")

    def predict(self, X):
        yHat = self.forward(X)
        return yHat
    def set_learning_rate(self, learning_rate):
        self.learning_rate = learning_rate

    def set_activation_function(self, activation_function):
        if(activation_function == 'sigmoid'):
            self.activation = self.sigmoid
            self.activationPrime = self.sigmoidPrime
        if(activation_function == 'relu'):
            self.activation = self.relu
            self.activationPrime = self.reluPrime
        if(activation_function == 'tanh'):
            self.activation = self.tanh
            self.activationPrime = self.tanhPrime

    def set_optimizer(self, optimizer):
        self.optimizer = optimizer

    def set_hidden_layers(self, hidden_layer_sizes):
        self.hiddenLayerSize = hidden_layer_sizes
        self.num_layers = len(self.hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        # self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
        # self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
        # self.weights, self.biases = self.initialize_weights()
    def set_epochs(self, epcohs):
        self.max_iterations = epochs
        print(self.max_iterations)






In [11]:
data = pd.read_csv('HousingData.csv')
imputer = SimpleImputer(strategy='mean')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
data = data.fillna(data.mean())
X = data.drop(columns=['MEDV'])
y = data['MEDV']
# X = data.iloc[1:, :-1].reset_index(drop=True)
# y = data.iloc[1:, -1].reset_index(drop=True)

# scaler = StandardScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)




In [12]:
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

X_train_std = std_scaler.fit_transform(X_train)
X_val_std = std_scaler.transform(X_val)
X_test_std = std_scaler.transform(X_test)

X_train_norm = minmax_scaler.fit_transform(X_train_std)
X_val_norm = minmax_scaler.transform(X_val_std)
X_test_norm = minmax_scaler.transform(X_test_std)
X_train = X_train_norm
X_val = X_val_norm
X_test = X_test_norm
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)
print(X_train)
print(y_train)

[[5.18056442e-04 3.47368421e-01 3.65623822e-02 ... 6.17021277e-01
  9.91833629e-01 1.43487859e-01]
 [2.97300779e-04 2.63157895e-01 1.37580098e-01 ... 6.80851064e-01
  9.84123764e-01 1.59492274e-01]
 [1.74592552e-03 2.31578947e-01 1.75273276e-01 ... 6.91489362e-01
  9.43723054e-01 2.14955850e-01]
 ...
 [6.68786251e-05 8.42105263e-01 3.01545420e-02 ... 4.68085106e-01
  9.84884606e-01 1.17549669e-01]
 [1.25342233e-01 0.00000000e+00 6.36637769e-01 ... 8.08510638e-01
  2.72001014e-01 5.94370861e-01]
 [2.46945108e-03 0.00000000e+00 2.77044855e-01 ... 8.82978723e-01
  1.72964748e-01 2.45584989e-01]]
[36.1 22.9 24.5 25.  50.  34.9 31.7 24.1 22.1 14.1 42.8 19.3 32.2 26.4
 21.8 21.7  8.3 46.7 43.1 31.5 10.5 16.7 20.  33.3 17.8 50.  20.5 23.2
 13.1 19.6 22.8 28.7 30.7 22.9 21.9 23.9 32.7 24.3 21.5 24.6  8.5 26.4
 23.1 15.   8.8 19.3 23.9 24.7 19.8 23.8 13.3 29.  27.1 34.6 13.3 15.6
 12.5 14.6 11.  24.8 17.3  8.1 21.4 15.6 23.3 32.  38.7 30.1 20.5 32.5
 42.3 24.3 20.6 22.  18.2 15.   6.3 20.1 21.4

In [15]:
def bagging_ensemble_regression(base_estimator, n_estimators, sample_fraction,with_replacement, voting):
    ensemble_models = []
    
    sample_size = int(sample_fraction * len(X_train))
    # np.random.seed(42)
    for _ in range(n_estimators):
        sample_indices = np.random.choice(len(X_train), size=sample_size, replace=with_replacement)
        sample_X = X_train[sample_indices]
        sample_y = y_train[sample_indices]
        sample_y = np.array(sample_y)
        if base_estimator == "DecisionTree":
            base_model = MyDecisionTreeRegressor()
            base_model.fit(sample_X, sample_y)
        elif base_estimator == "Linear":
            base_model = MyLinearRegressor()
            base_model.fit(sample_X, sample_y)
        elif base_estimator == "MLP":
            base_model = MyMLPRegressor()
            
            y_train_array = np.array([[x] for x in sample_y])
            base_model.fit(sample_X, y_train_array)
            

        # base_model.fit(sample_X, sample_y)
        ensemble_models.append(base_model)

    def ensemble_predict(X):
        predictions = [model.predict(X) for model in ensemble_models]

        if voting == "hard":
            return np.round(np.mean(predictions, axis=0))
        elif voting == "soft":
            confidences = [1 / (mean_squared_error(y_val, model.predict(X_val)) + 1e-10) for model in ensemble_models]  
            weighted_predictions = [prediction * confidence for prediction, confidence in zip(predictions, confidences)]
            return np.sum(weighted_predictions, axis=0) / np.sum(confidences)

    return ensemble_predict

bagged_regressor_decision_tree_soft = bagging_ensemble_regression(base_estimator="DecisionTree", n_estimators=10, sample_fraction=0.75, with_replacement=True, voting="soft")
y_pred = bagged_regressor_decision_tree_soft(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 11.196675712448528


# Hyperparameter Tuning

In [7]:
n_estimators = [1,5,10]
base_estimator = ["DecisionTree","Linear","MLP"]
sample_fraction = [0.15,0.25,0.5,0.75,1]
bootstrap = [True,False]
voting = ["soft","hard"]
best_mse = 10000000
best_params = []
for i in n_estimators:
    for j in base_estimator:
        for k in sample_fraction:
            for l in bootstrap:
                for m in voting:
                    bagged_regressor = bagging_ensemble_regression(base_estimator=j, n_estimators=i, sample_fraction=k, with_replacement=l, voting=m)
                    y_pred = bagged_regressor(X_test)
                    mse = mean_squared_error(y_test, y_pred)
                    print("Mean Squared Error:", mse)
                    print("n_estimators:",i)
                    print("base_estimator:",j)
                    print("sample_fraction:",k)
                    print("bootstrap:",l)
                    print("voting:",m)
                    print("-----------------------------------------------------")
                    if(mse < best_mse):
                        best_mse = mse
                        best_params = [i,j,k,l,m]
print("Best MSE:",best_mse)
print("Best Parameters:",best_params)
                    

Mean Squared Error: 37.896960784313734
n_estimators: 1
base_estimator: DecisionTree
sample_fraction: 0.15
bootstrap: True
voting: soft
-----------------------------------------------------
Mean Squared Error: 57.476078431372564
n_estimators: 1
base_estimator: DecisionTree
sample_fraction: 0.15
bootstrap: True
voting: hard
-----------------------------------------------------
Mean Squared Error: 25.440294117647063
n_estimators: 1
base_estimator: DecisionTree
sample_fraction: 0.15
bootstrap: False
voting: soft
-----------------------------------------------------
Mean Squared Error: 34.552549019607845
n_estimators: 1
base_estimator: DecisionTree
sample_fraction: 0.15
bootstrap: False
voting: hard
-----------------------------------------------------
Mean Squared Error: 37.91137254901961
n_estimators: 1
base_estimator: DecisionTree
sample_fraction: 0.25
bootstrap: True
voting: soft
-----------------------------------------------------
Mean Squared Error: 24.570196078431376
n_estimators: 1

KeyboardInterrupt: 

In [16]:
data = pd.read_csv('WineQT.csv', header=0)
train_data, test_data = train_test_split(data, test_size=0.2,random_state = 42)

scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

imputer = SimpleImputer(strategy='mean')  

train_data_scaled = scaler.fit_transform(imputer.fit_transform(train_data.drop(columns=['quality'])))
train_labels = train_data['quality']

test_data_scaled = scaler.transform(imputer.transform(test_data.drop(columns=['quality'])))

train_data_scaled_normalized = minmax_scaler.fit_transform(train_data_scaled)
test_data_scaled_normalized = minmax_scaler.transform(test_data_scaled)

X_train = train_data_scaled
y_train = train_labels
X_test = test_data_scaled
y_test = test_data['quality']
y_train = np.array(y_train)
y_test = np.array(y_test)
# y_train = pd.get_dummies(y_train).astype(int)
# y_train = np.eye(9)[y_train]
# y_train = np.array(y_train)
# y_test = pd.get_dummies(y_test).astype(int)
# print(X_train)
# print(y_train)
# y_test = np.eye(9)[y_test]
# y_test = np.array(y_test)
y_test = np.eye(9)[y_test]
y_test = np.array(y_test)


In [41]:
def bagging_ensemble_classification(base_classifier, n_estimators, sample_fraction, with_replacement, voting):
    ensemble_models = []
    
    sample_size = int(sample_fraction * len(X_train))
    # np.random.seed(42)
    for _ in range(n_estimators):
        sample_indices = np.random.choice(len(X_train), size=sample_size, replace=with_replacement)
        sample_X = X_train[sample_indices]  # Assuming X_train is a DataFrame
        sample_y = y_train[sample_indices]  # Assuming y_train is a Series

        if base_classifier == "DecisionTree":
            base_model = MyDecisionTreeClassifier()
            sample_y = np.eye(9)[sample_y]
            sample_y = np.array(sample_y)
            base_model.fit(sample_X, sample_y)
        elif base_classifier == "Logistic":
            base_model = MyLogisticClassifier()
            base_model.fit(sample_X, sample_y)
        elif base_classifier == "MLP":
            base_model = MyMLPClassifier()
            sample_y = np.eye(9)[sample_y]
            sample_y = np.array(sample_y)
            base_model.fit(sample_X, sample_y)
            
        ensemble_models.append(base_model)

    def ensemble_predict(X):
        predictions = [model.predict(X) for model in ensemble_models]
        predictions = np.array(predictions)
        if(base_classifier == "DecisionTree"):
            if voting == "hard":
                array_2d = np.sum(predictions, axis=0)
                for array in range(len(array_2d)):
                    k = np.argmax(array_2d[array])
                    array_2d[array] = np.zeros(9)
                    array_2d[array][k] = 1
                return array_2d

            elif voting == "soft":
                confidences = [accuracy_score(y_test, model.predict(X_test)) for model in ensemble_models]
                confidences = confidences / sum(confidences)
                print(confidences)
                for i in range(len(confidences)):
                    predictions[i] = predictions[i] * confidences[i]
                array_2d = np.sum(predictions, axis=0)
                for array in range(len(array_2d)):
                    k = np.argmax(array_2d[array])
                    array_2d[array] = np.zeros(9)
                    array_2d[array][k] = 1
                return array_2d
        elif(base_classifier == "Logistic"):
            if voting == "hard":
                array_2d = np.zeros((229, 9))
                for idx in range(len(predictions[0])):
                    uniq, cnt = np.unique(predictions[:, idx], return_counts=True)
                    k = np.argmax(cnt)
                    array_2d[idx][uniq[k]] = 1
                return array_2d

            elif voting == "soft":
                array_2d = np.zeros((229,9))
                y_test_copy = y_test.copy()
                confidences = [accuracy_score(np.argmax(y_test,axis=1), model.predict(X_test)) for model in ensemble_models]
                print(confidences)
                confidences = confidences / sum(confidences)
                for idx in range(len(predictions[0])):
                    uniq = np.zeros(9)
                    fin = 0
                    wei = 0
                    for i in range(len(confidences)):
                        uniq[predictions[i][idx]] += confidences[i]
                        if(wei < uniq[predictions[i][idx]]):
                            wei = uniq[predictions[i][idx]]
                            fin = predictions[i][idx]
                    array_2d[idx][fin] = 1
                return array_2d
        elif(base_classifier == "MLP"):
            if voting == "hard":
                array_2d = np.sum(predictions, axis=0)
                for array in range(len(array_2d)):
                    k = np.argmax(array_2d[array])
                    array_2d[array] = np.zeros(9)
                    array_2d[array][k] = 1
                return array_2d

            elif voting == "soft":
                confidences = [accuracy_score(y_test, model.predict(X_test)) for model in ensemble_models]
                confidences = confidences / sum(confidences)
                print(confidences)
                for i in range(len(confidences)):
                    predictions[i] = predictions[i] * confidences[i]
                array_2d = np.sum(predictions, axis=0)
                for array in range(len(array_2d)):
                    k = np.argmax(array_2d[array])
                    array_2d[array] = np.zeros(9)
                    array_2d[array][k] = 1
                return array_2d

    return ensemble_predict

# Example usage for bagging ensemble with Decision Tree classifier and soft voting
bagged_classifier_decision_tree_soft = bagging_ensemble_classification(base_classifier="Logistic", n_estimators=10, sample_fraction=0.25, with_replacement=True, voting="soft")

# Make predictions using the bagged ensemble
y_pred = bagged_classifier_decision_tree_soft(X_test)
y_pred = np.array(y_pred)
print(y_test.shape)
print(y_pred.shape)
for i in range(len(y_pred)):
    print(y_pred[i])
    print(y_test[i])
print(y_pred[1])
y_test = np.array(y_test)
y_pred = np.array(y_pred)
# print("Pred :", y_pred)
# print("Test :", y_test)
# Evaluate the performance, e.g., by calculating the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[0.6157205240174672, 0.611353711790393, 0.6331877729257642, 0.5982532751091703, 0.62882096069869, 0.62882096069869, 0.5807860262008734, 0.5764192139737991, 0.6157205240174672, 0.5720524017467249]
[0.        0.        0.        0.        0.        0.8962536 0.1037464
 0.        0.       ]
5
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
5
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
5
[0.         0.         0.         0.         0.         0.69596542
 0.20244957 0.10158501 0.        ]
5
[0.         0.         0.         0.         0.         0.
 0.79971182 0.20028818 0.        ]
6
[0.        0.        0.        0.        0.        0.        0.1981268
 0.8018732 0.       ]
7
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
5
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
5
[0.         0.         0.         0.         0.         0.30115274
 0.69884726 0.         0.        ]
6
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
5
[0.         0.         0.         0.         0.         0.
 0.10086455 0.89913545 0.        ]
7
[0.         0.         0.         0.         0.         0.
